In [1]:
import os

from pydantic import BaseModel,Field
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# OpenAI api define
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model="gpt-4o",base_url=os.getenv("base_url"))

In [3]:
# Pydatntic class definition
class EntityExtraction(BaseModel):
    entities:str=Field(description="all the entities in a provided content")

In [9]:
entity_structured_output_model = model.with_structured_output(schema=EntityExtraction)

In [11]:
entities_output=entity_structured_output_model.invoke([{"role":"system","content":"extract named entities from the given content"},{"role":"user","content":"In New York City, the iconic statue of Liberty stands tall as a symbol of freedom, attracting millions of tourists annually. The historic monument, gifted by France in 1886, is located on Liberty Island and overlooks the bustling Hudson River. Managed by the National Park Service, the site offers guided tours and educational programs, providing visitors with insights into American history and culture."}])

In [12]:
entities_output.entities

'New York City, Statue of Liberty, France, 1886, Liberty Island, Hudson River, National Park Service, American history and culture'

In [7]:
class NER_structure(BaseModel):
    Subject:str=Field(description="The entity being described.")
    Predicate:str=Field(description="The property or relationship associated with the subject.") 
    Object:str=Field(description=" The value or entity to which the predicate applies.")

In [8]:
class EntityAndRelationship_Extraction(BaseModel):
    relationship:list[NER_structure] = Field(description ="these are the relationship between extracted named entities")

In [13]:
relationship_structured_output_model = model.with_structured_output(schema=EntityAndRelationship_Extraction)

In [14]:
relationship_output=relationship_structured_output_model.invoke([{"role":"system","content":"extract the relationship between identified named entities from the given content"},{"role":"user","content":f"In New York City, the iconic statue of Liberty stands tall as a symbol of freedom, attracting millions of tourists annually. The historic monument, gifted by France in 1886, is located on Liberty Island and overlooks the bustling Hudson River. Managed by the National Park Service, the site offers guided tours and educational programs, providing visitors with insights into American history and culture. {entities_output.entities}"}])

In [15]:
relationship_output

EntityAndRelationship_Extraction(relationship=[NER_structure(Subject='Statue of Liberty', Predicate='located in', Object='New York City'), NER_structure(Subject='Statue of Liberty', Predicate='symbolizes', Object='freedom'), NER_structure(Subject='Statue of Liberty', Predicate='gifted by', Object='France'), NER_structure(Subject='Statue of Liberty', Predicate='gifted in', Object='1886'), NER_structure(Subject='Statue of Liberty', Predicate='situated on', Object='Liberty Island'), NER_structure(Subject='Statue of Liberty', Predicate='overlooks', Object='Hudson River'), NER_structure(Subject='Statue of Liberty', Predicate='managed by', Object='National Park Service'), NER_structure(Subject='National Park Service', Predicate='offers', Object='guided tours and educational programs'), NER_structure(Subject='guided tours and educational programs', Predicate='provide insights into', Object='American history and culture')])